# Second lecture: Julia

### Benchmarking Julia

We will see the implementation $ benchmarking of **sum function** in:

* Julia (built-in)  
* Julia (hand-written)  
* C (hand-written)  
* python (built-in)  
* python (numpy)  
* python (hand-written)  

In [5]:
import Pkg
Pkg.instantiate()

In [2]:
a = rand(10^7) # vector of random numbers, uniform on [0,1]

10000000-element Vector{Float64}:
 0.2525576338368869
 0.459376587627343
 0.2637048375367672
 0.7638436869058564
 0.8821751788959045
 0.42556199238688197
 0.9332979534403669
 0.8777217322472689
 0.19977933798653824
 0.8448964080507566
 0.4179522558504025
 0.384310657576593
 0.7223376034204833
 ⋮
 0.5171675902611497
 0.7710086786911577
 0.7971251800186492
 0.5562411216123363
 0.4959889600208085
 0.7560780458271646
 0.2134001807257433
 0.10569240732818519
 0.14980641889900081
 0.18179281210167364
 0.5873474351775368
 0.35527807915897935

In [3]:
@which sum(a)

sum(a::AbstractArray; dims, kw...)
     @ Base reducedim.jl:1010

In [4]:
sum(a)

5.000948072351623e6

In [97]:
@time sum(a)  # try to repeat the execution of this cell!

  0.003172 seconds (1 allocation: 16 bytes)


5.000948072351623e6

but why do we get different values of the computing time?


In [98]:
Pkg.add("BenchmarkTools")
using BenchmarkTools

   Resolving package versions...
   Installed BenchmarkTools ─ v1.5.0
    Updating `~/.julia/environments/v1.10/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.5.0
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.5.0
  [9abbd945] + Profile
Precompiling project...
  ✓ BenchmarkTools
  1 dependency successfully precompiled in 2 seconds. 227 already precompiled.


In [99]:
@benchmark sum(a)

BenchmarkTools.Trial: 1737 samples with 1 evaluation.
 Range (min … max):  2.692 ms …   4.623 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.705 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.869 ms ± 253.969 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▄▂                         ▂▄     ▂▁▁                       
  ███▇▆▆▇▆▆▅▇▄▇▇▅▇▇▆▅▄▅▅▅▇▆▆▅▆████▇▇█████▇▇▆▇▇▇▆▇▅▆▄▃▅▄▅▃▁▁▄▅ █
  2.69 ms      Histogram: log(frequency) by time      3.55 ms <

 Memory estimate: 16 bytes, allocs estimate: 1.

If the expression to benchmark depends on external variables, one should use `$` to "interpolate" them into the benchmark expression to avoid the problems of benchmarking with globals. Essentially, any interpolated variable `$x` or expression `$(...)` is "pre-computed" before benchmarking begins. So in short with `@btime` `$` is used to "interpolate" them into the benchmarked expression in order to get a correct benchmark results.

In this way we pre-compute the value of `a` and then execute the benchmark:

In [100]:
@benchmark sum($a)

BenchmarkTools.Trial: 1707 samples with 1 evaluation.
 Range (min … max):  2.692 ms …   4.781 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.720 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.922 ms ± 269.846 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▄▁                      ▅▂▃▁ ▃▁▂                            
  ███▇▇▇▅▆▇▆▆▇▆▇▆▅▇▅▆▆▆▆▅▅████████████▆▆▇▇▆▇▇▅▆▄▃▃▄▁▅▃▁▄▃▄▄▄▆ █
  2.69 ms      Histogram: log(frequency) by time      3.68 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [103]:
j_bench = @benchmark sum($a)

d = Dict()
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any, Any} with 1 entry:
  "Julia built-in" => 2.68992

Let us implement our hand-written sum function:

In [104]:
function mysum(A)
    s = 0.0
    for a in A
        s += a
    end
    return s
end

mysum (generic function with 1 method)

In [105]:
j_bench_hand = @benchmark mysum($a)
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any, Any} with 2 entries:
  "Julia hand-written" => 12.8121
  "Julia built-in"     => 2.68992

Let us now compare it to the C language, that we can use inside the Julia environment:

In [108]:
using Libdl
C_code = """
    #include <stddef.h>
    double c_sum(size_t n, double *X) {
        double s = 0.0;
        for (size_t i = 0; i < n; ++i) {
            s += X[i];
        }
        return s;
    }
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code)
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [109]:

# Define the C++ function and compile it
C_code = """
    #include <stddef.h>
    double c_sum(size_t n, double *X) {
        double s = 0.0;
        for (size_t i = 0; i < n; ++i) {
            s += X[i];
        }
        return s;
    }
"""

# Execute it
icxx"myCppFunction();" # Return "Printing 10"
# OR
# Convert the C++ to Julia function
myJuliaFunction() = @cxx myCppFunction()
# Run the function
myJuliaFunction() # Return "Printing 10"

LoadError: LoadError: UndefVarError: `@icxx_str` not defined
in expression starting at In[109]:15

In [110]:
c_sum(a)
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbol

true

In [111]:
c_bench = @benchmark c_sum($a)
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

Dict{Any, Any} with 3 entries:
  "C"                  => 12.7635
  "Julia hand-written" => 12.8121
  "Julia built-in"     => 2.68992

let us see the time of the same operation, but using python. we can do it with the package PyCall.

In [113]:
Pkg.add("PyCall")
using PyCall

   Resolving package versions...
   Installed PyCall ─ v1.96.4
    Updating `~/.julia/environments/v1.10/Project.toml`
  [438e738f] + PyCall v1.96.4
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [438e738f] + PyCall v1.96.4
    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/9816a3826b0ebf49ab4926e2b18842ad8b5c8f04/build.log`
Precompiling project...
  ✓ PyCall
  1 dependency successfully precompiled in 9 seconds. 228 already precompiled.


In [114]:
pysum = pybuiltin("sum")
pysum(a)

py_list_bench = @benchmark $pysum($a)
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any, Any} with 4 entries:
  "C"                  => 12.7635
  "Julia hand-written" => 12.8121
  "Julia built-in"     => 2.68992
  "Python built-in"    => 702.062

let us check the python numpy sum function:

In [115]:
np=pyimport("numpy")
np.sum(a)

py_numpy = @benchmark $np.sum($a)
d["Python numpy"] = minimum(py_numpy.times) / 1e6
d

Dict{Any, Any} with 5 entries:
  "C"                  => 12.7635
  "Julia hand-written" => 12.8121
  "Python numpy"       => 2.979
  "Julia built-in"     => 2.68992
  "Python built-in"    => 702.062

In [116]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

Julia built-in..............2.7
Python numpy................3.0
C..........................12.8
Julia hand-written.........12.8
Python built-in...........702.1


apparently, Julia is the fastest! measure units are milliseconds. but we must admit that here we are using C hand-written sum (using for cycle), so we can assume that python numpy (written in C) has the same time of the optimized C code.

# Serial code optimization

Several decisions can be considered while designing the code (e.g. Types, Type Inference, and Stability, multiple dispatch, meta-programming, vectorization, ...). Such methodologies allow to write an efficient computer program for high performance computing.

Before writing any fast parallel code, one should consider write a fast serial code. Here are some tips that could make your serial code run faster:  
1. __Write your performance critical code inside a function__  
    Code inside functions tends to run much faster than top level code, due to how Julia's compiler works. Functions not only enhances the performance but it is more reusable and testable.  
2. __Make variables local__
    A global variable could change its type during execution, so it could be difficult for the julia compiler optimize it. Using local variable instead or declaring the global variable as constant (`const:`) will greatly improve performances.
3. __Use `@time` or `@btime`__  
    Use `@time` or `@btime` macro to measure the execution time of a function. They can also indicate the amount of allocated which could be sometimes significant.  
4. __Declare types when possible__  
    Type stability makes for loops faster. Declaring types help compilers to convert into machine code.

In [117]:
function xpow(x)
    return [x x^2 x^3 x^4]
end

xpow (generic function with 1 method)

In [118]:
function xpow_loop(n)
    s= 0
    for i = 1:n
        s = s + xpow(i)[2]
    end
    return s 
end

xpow_loop (generic function with 1 method)

In [119]:
using BenchmarkTools; @btime xpow_loop(1000000)

  34.836 ms (1000000 allocations: 91.55 MiB)


333333833333500000

# Julia Array-type optimization  
Arrays are a fundamental data structure in all programming languages. Moreover, they can improve performance. Thus, a special attention is given to the usage of array in numerical programming.

We will discuss how to use arrays in Julia in the most efficient way:
- Computer memory model and array representation and storage in Julia
- Bounds checks and **@inbounds**
- Specialized array types
- __Broadcasting__
- __SIMD__ parallelization

Julia implement column major ordering, just like Fortran, Matlab, R. On the other hand, arrays in C/C++ and Python's numpy are stored as row major-ordered.  
The following code squares and sums the elements of a two-dimensional floating point array, writing the result at each step back to the same position. The following code exercises both the read and write operations for the array:

In [120]:
function col_iter(x)
    s=zero(eltype(x))
    for i in 1:size(x, 2)
        for j in 1:size(x, 1)
            s = s + x[j, i] ^ 2
            x[j, i] = s
        end
    end
end

col_iter (generic function with 1 method)

In [121]:
function row_iter(x)
    s=zero(eltype(x))
    for i in 1:size(x, 1)
        for j in 1:size(x, 2)
            s = s + x[i, j] ^ 2
            x[i, j] = s
        end
    end
end

row_iter (generic function with 1 method)

In [122]:
a = rand(1000, 1000)
@btime col_iter(a)
@btime row_iter(a)

  1.238 ms (0 allocations: 0 bytes)
  1.464 ms (0 allocations: 0 bytes)


### Lower Level View: the Stack and the Heap

**Stack**  
- The stack requires a static allocation. It is ordered and accesses are very quick.
- Because this is static, it requires that the size of the variables is known at compile time (to determine all of the variable locations).
    
**Heap**  
- The heap is essentially a stack of pointers to objects in memory. When heap variables are needed, their values are pulled up the cache chain and accessed.
- Heap allocations are costly because they involve this pointer dereferencing.

In [123]:
a = Int64[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
b = Number[1,2,3,4,5,6,7,8,9,10]  #Number is a supertype of Int64

10-element Vector{Number}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [124]:
function arr_sumsqr(x::Array{T}) where T <: Number  # with <: T inherits Number 
    r = zero(T)
    for i = 1:length(x)
        r = r + x[i] ^ 2
    end
    return r
end

arr_sumsqr (generic function with 1 method)

In [125]:
@btime arr_sumsqr($a)
@btime arr_sumsqr($b)

  6.541 ns (0 allocations: 0 bytes)
  486.323 ns (0 allocations: 0 bytes)


385

When the array is defined to contain a specific concrete type, the Julia runtime can store the values inline within the allocation of the array, since it knows the exact size of each element. When the array contains an abstract type, the actual value can be of any size. Thus, when the Julia runtime creates the array, it only stores the pointers to the actual values within the array. The values are stored elsewhere on the heap. This not only causes extra memory load when reading the values, but the indirection can mess up pipelining and cache affinity when executing this code on the CPU.

Consider `StaticArrays.jl` for small fixed-size vector/matrix operations.  
- StaticArrays.jl library uses statically-sized arrays and thus arrays which are stack-allocated. 
- It can be used for many small (< 100 element) arrays of fixed sizes.

In [126]:
Pkg.add("StaticArrays")

   Resolving package versions...
    Updating `~/.julia/environments/v1.10/Project.toml`
  [90137ffa] + StaticArrays v1.9.3
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [128]:
A = rand(100,100)
B = rand(100,100)
C = rand(100,100)

using StaticArrays
function static_inner_alloc!(C,A,B)
    for j in 1:100, i in 1:100
        val = @SVector [A[i,j] + B[i,j]]
        C[i,j] = val[1]
    end
end
@btime static_inner_alloc!(C,A,B)

  3.885 μs (0 allocations: 0 bytes)


## Bounds checks 
Julia performs bounds checks on arrays by default at runtime. This means that the Julia compiler and runtime verify that the arrays are not indexed outside their limits and that all the indexes lie between the actual start and end of an array.

The `@inbounds` macro eliminates array bounds checking within expressions and is applied in front of a function or loop definition.

In [129]:
len = (100, 100);
a, b = fill(1.0f0, len), fill(2.0f0, len);

Using `fill` in this way is not only convenient but also safer, since the memory that is returned to the program is filled with known good values. However, the operation to fill the memory is also expensive. For a performance-critical method to create arrays, the constructor can be called with a special `undef` keyword. In this case, memory is allocated, but not filled.

In [130]:
function prefix_bounds(a,b)
    c = similar(a)
    for j in 1:100, i in 1:100
        c[i,j] = a[i,j] + b[i,j]
    end
end

@btime prefix_bounds($a, $b)
@code_llvm prefix_bounds(a,b)

  9.875 μs (2 allocations: 39.17 KiB)
;  @ In[130]:1 within `prefix_bounds`
define void @julia_prefix_bounds_5452({}* noundef nonnull align 16 dereferenceable(40) %0, {}* noundef nonnull align 16 dereferenceable(40) %1) #0 {
top:
;  @ In[130]:2 within `prefix_bounds`
; ┌ @ array.jl:416 within `similar`
; │┌ @ array.jl:190 within `size`
    %2 = bitcast {}* %0 to {}**
    %arraysize_ptr = getelementptr inbounds {}*, {}** %2, i64 3
    %3 = bitcast {}** %arraysize_ptr to i64*
    %arraysize = load i64, i64* %3, align 8
    %arraysize_ptr1 = getelementptr inbounds {}*, {}** %2, i64 4
    %4 = bitcast {}** %arraysize_ptr1 to i64*
    %arraysize2 = load i64, i64* %4, align 8
; │└
; │┌ @ boot.jl:479 within `Array`
    %5 = call nonnull {}* inttoptr (i64 4387225500 to {}* ({}*, i64, i64)*)({}* inttoptr (i64 4534035024 to {}*), i64 %arraysize, i64 %arraysize2)
    %6 = bitcast {}* %0 to float**
    %7 = bitcast {}* %1 to {}**
    %arraysize_ptr11 = getelementptr inbounds {}*, {}** %7, i64 3
  

In [131]:
function prefix_bounds(a,b)
    c = similar(a)
    @inbounds for j in 1:100, i in 1:100
        c[i,j] = a[i,j] + b[i,j]
    end
end

@btime prefix_bounds($a, $b)

  1.744 μs (2 allocations: 39.17 KiB)


When the Julia environment is started with `-check-bounds=yes`, all @inbounds annotations in the code are ignored, and bounds checks are mandatorily performed. Alternatively, when the Julia runtime is started with `-check-bounds=no`, no bound checking is done at all. This is equivalent to annotating all array access with the `@inbounds` macro. This option should only be used sparingly in the case of extremely performancesensitive code, in which the system is very well tested and with minimal user inputs.

## Broadcasting

Performs an operation on each element of an array, rather than on the array as a whole. In many high level languages this is simply called vectorization.  
*In Julia, we will call it array vectorization to distinguish it from the SIMD vectorization which is common in lower level languages like C, Fortran, and Julia.*

In [132]:
a=collect(1:4)

4-element Vector{Int64}:
 1
 2
 3
 4

In [133]:
sqrt.(a)

4-element Vector{Float64}:
 1.0
 1.4142135623730951
 1.7320508075688772
 2.0

### Exercises

Using these libraries:

```julia
using BenchmarkTools, Test, SIMD, InteractiveUtils
```

### **Q1**  
Consider the following function that sums all the elements of a vector:  

```julia
A = rand(__put a BIGNUMBER here__)  

function simplesum(A)  
    result = zero(eltype(A))  
    for i in eachindex(A)  
        result += A[i]  
    end  
    return result  
end  
```   

- Try to optimize it for bounds checks and benchmark against the in-built julia function `sum`
- Implement SIMD parallelization and compute the obtained speedup

In [147]:
Pkg.add("SIMD")
using SIMD

   Resolving package versions...
   Installed SIMD ─ v3.4.6
    Updating `~/.julia/environments/v1.10/Project.toml`
  [fdea26ae] + SIMD v3.4.6
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [fdea26ae] + SIMD v3.4.6
Precompiling project...
  ✓ SIMD
  1 dependency successfully precompiled in 8 seconds. 229 already precompiled.


In [148]:
using BenchmarkTools, SIMD

A = rand(10^8)  

function simplesum(A)
    result = zero(eltype(A))  
    for i in eachindex(A)  
        @inbounds result += A[i]  
    end  
    return result  
end

simplesum (generic function with 1 method)

In [149]:
builtin_bench = @benchmark sum($A)
my_bench = @benchmark simplesum($A)

println("My simplesum minimum time = ", minimum(my_bench.times) / 1e6)
println("Built-in minimum time = ", minimum(builtin_bench.times) / 1e6)

My simplesum minimum time = 144.027083
Built-in minimum time = 27.758167


In [150]:
serial_t = @belapsed simplesum(A)

function simdsum(A)
    result = zero(eltype(A))
    @simd for i in eachindex(A)
        @inbounds result += A[i]
    end
    return result
end

simd_t = @belapsed simdsum(A)

times = [serial_t, simd_t]
speedup = maximum(times) ./ times

2-element Vector{Float64}:
 1.0
 3.992370837344596

### **Q2**

The following function estimates pi with n samples. 

    - Optimize it for bounds checks
    - Use `time()` function in order to measure elapsed execution time for the function. If you don't know ho to use it, why not typing `?time()` ?
    - Parallelize the code in order to exploit SIMD and measure elapsed execution time
    - Compute relative error with respect to true Julia value of `PI` and compare the one obtained from SIMD version with the no-SIMD version

In [161]:
n = 10^7

10000000

In [168]:
function estimatepi(n)
    area_circle = 0
    for i = 1:n
        x, y = rand(), rand();
        r = x^2 + y^2
        if r < 1.0
            area_circle +=1
        end
    end
    return 4* area_circle/n
end

@benchmark estimatepi(10^7)

BenchmarkTools.Trial: 128 samples with 1 evaluation.
 Range (min … max):  38.747 ms …  40.771 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     39.180 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   39.275 ms ± 333.704 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

          ▄▇▄██▆▄                                               
  ▃▁▃▁▃▁▄▅███████▅█▆▆▇▆▅▃▃▃▄▁▁▁▁▁▁▁▁▃▁▁▃▄▁▁▃▃▁▃▁▁▁▁▁▁▁▁▁▁▃▁▁▁▃ ▃
  38.7 ms         Histogram: frequency by time         40.7 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [170]:
function modified_estimatepi(n)
    area_circle = 0
    @simd for i = 1:n
        x, y = rand(), rand();
        r = x^2 + y^2
        if r < 1.0
            @inbounds area_circle +=1
        end
    end
    return 4* area_circle/n
end

t = @belapsed modified_estimatepi(10^7)

0.038839375

## Last exercise

TODO

# CPU parallelism

A modern computer can appear to do many things in parallel. It can play music while you are writing a report, run your tests when you are typing code, or run a web server in the background. There are many layers to achieving this level of multiprocessing in Julia.

Julia exhibits an excellent productivity as high-level programming language without sacrificing performance. In fact it:
- is a dynamically typed high-level languague
- has a built-in packages manager
- has an interactive development
- has JIT-compilation
- has reflection and metaprogramming
- has great performances
Moreover Julia avoids runtime uncertainties thanks to its sophisticated type system, multiple dispatch and type interference. Nonetheless most of the Julia code is written in Julia, functions are generic and dynamic since types are inferred. Last, but not least, LLVM code is generated by JIT-compiler, which is then optimized and compiled to native code.

Topics covered in this section (and on following notebooks):
* __Elements of supercomputing and parallel programming__ 
* __Julia parallel computing__:
    * SIMD
    * Tasks
    * Multi-threading
    * Distributed computing
    * GPU computing

An algorithm that uses only a single core is referred to as a __serial algorithm__. These algorithms complete one instruction at a time, in order. 

Parallelization means converting a serial algorithm that can perform multiple operations simultaneously.  

Most programs in Julia run on a single thread, on a single processor core. In other words, most processing in Julia is __synchronous__. 

In [171]:
#How long does this take?
@time for i in 1:5
    sleep(1)
end

  5.011192 seconds (124 allocations: 3.297 KiB)


In [172]:
@time for i in 1:5
    @async sleep(1) # creates and schedules tasks for all code within its scope
end

  0.011853 seconds (2.82 k allocations: 203.891 KiB, 96.87% compilation time)


In [174]:
@time @sync for i in 1:5
    @async sleep(1)
end

  1.007544 seconds (7.84 k allocations: 556.898 KiB, 0.48% compilation time)


The `@async` macro creates and schedules tasks for all code within its scope, which means all the sleep calls happen in separate tasks. The for loop, which is running on the REPL task, returns almost immediately. Since the for loop returns, we cannot even observe the sleep occurring. It happens completely in the background.

`@sync` macro will pause a task until all tasks created by it have finished execution. 

This parallelism is somewhat simulated. While tasks can be interrupted and switch between one another, at any one time, only one task is running on the CPU. Thus, you will see a performance benefit only for code that is doing something but not taking CPU resources: this typically means it is waiting for some I/O, or as in the degenerate case previously, sleeping. Hence this pattern of code is useful in programs that are I/O heavy—where code spends a lot of time waiting for disk or network data.

## Julia multithreading with `Threads`  

Threads are sequences of computation that can run independently on a CPU core, simultaneously with other such sequences. Julia uses a task-based model, where we have a fixed number of threads and schedule defined pieces of work onto them.
- Unlike tasks, which are lightweight, threads need to store some state when they are switched  
- While you can have hundreds or thousands of tasks in your program, you should only have a limited number of threads. The general advice is that the number of threads should correspond directly to the number of CPU cores you have  
- Share the same memory space

**They have some cost:**
- Synchronisation issue: multiple threads can be accessed by the same variable.
- All threads must live in the same physical machine.

### Key terminology
- __Tasks__: computational tasks such as function  
- __Spawning a task__: assigning a computational task on different threads that run simultaneously  
- __Workload__: amount of work required to compute a task  
- __Composable task parallelism__: a task can spawn new tasks  

how many physical core do we have?

In [177]:
versioninfo(verbose=true)

Julia Version 1.10.1
Commit 7790d6f0641 (2024-02-13 20:41 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  uname: Darwin 23.0.0 Darwin Kernel Version 23.0.0: Fri Sep 15 14:42:57 PDT 2023; root:xnu-10002.1.13~1/RELEASE_ARM64_T8112 arm64 arm
  CPU: Apple M2: 
              speed         user         nice          sys         idle          irq
       #1  2400 MHz      94751 s          0 s      64650 s     881036 s          0 s
       #2  2400 MHz      87445 s          0 s      58101 s     895409 s          0 s
       #3  2400 MHz      76445 s          0 s      47776 s     917746 s          0 s
       #4  2400 MHz      68290 s          0 s      40273 s     934200 s          0 s
       #5  2400 MHz     246405 s          0 s      15551 s     782358 s          0 s
       #6  2400 MHz     236674 s          0 s      11659 s     796170 s          0 s
       #7  2400 MHz     224833 s          0 s       7866 s     811975 s          

## Starting and using `threads`

The number of real threads that Julia can run is fixed at startup. It must be set on `JULIA_NUM_THREADS` environmental variable and is checked when the Julia runtime starts up. If the variable is not set, the default number of threads is 1.  

1) Open a terminal and write `export JULIA_NUM_THREADS = 4`
2) Open Julia, or if you want to use an IDE (such as VSCode), type `code .`
3) Once Julia kernel is started, you can check the number of threads using the `nthreads()` function. 


In [6]:
Pkg.add("Hwloc")
using Hwloc

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [7]:
Hwloc.num_physical_cores()

8

In [10]:
using Base.Threads, BenchmarkTools

# squential 
for i = 1:8
    println("iteration $i Hello Julia from thread ", threadid())
  end 

nthreads()

iteration 1 Hello Julia from thread 1
iteration 2 Hello Julia from thread 1
iteration 3 Hello Julia from thread 1
iteration 4 Hello Julia from thread 1
iteration 5 Hello Julia from thread 1
iteration 6 Hello Julia from thread 1
iteration 7 Hello Julia from thread 1
iteration 8 Hello Julia from thread 1


4

In [11]:
#The @threads macro
    @threads for i = 1:8
        println("iteration $i Hello Julia from thread ", threadid())
      end  

iteration 5 Hello Julia from thread 3
iteration 1 Hello Julia from thread 1
iteration 2 Hello Julia from thread 2
iteration 7 Hello Julia from thread 2
iteration 3 Hello Julia from thread 4
iteration 6 Hello Julia from thread 3
iteration 8 Hello Julia from thread 2
iteration 4 Hello Julia from thread 4


`@spawn` macro runs a function on some available thread. This is an example of __non-blocking__ function. `fetch` waits for a Task to finish, then return its result value. 

In [45]:
@spawn println("Hello Julia from thread ", threadid()) 

Hello Julia from thread 1


Task (done) @0x0000000294d54e20

In [46]:
for i in 1:30
    print(fetch(@spawn threadid()), " ")
end

2 1 1 1 1 1 2 2 4 3 4 3 4 3 2 3 2 3 2 4 3 4 4 4 2 2 1 2 2 1 

we are not seeing the thread number 1, since it is now working to show the others thread!

## Thread safety
Threads imply code running simultaneously on multiple processor cores in a computer. The processors and the code running within them have access to the entire memory of the computer. This means that code in two threads can attempt to change the same piece of data in memory at the same time. This is called __race condition__. The result of an operation can be wrong and random at each execution and a multithreaded function can also be slower than the serial one.

In [47]:
function sum_thread_serial(x)
    r = zero(eltype(x))
    for i in eachindex(x)
        @inbounds r += x[i]
    end
    return r 
end

sum_thread_serial (generic function with 1 method)

In [64]:
sum_thread_serial(rand(100000))

50019.80897738643

One solution to make `r` as an __atomic__ variable. Adding a value to r will be a single indivisible operation and only one thread at the time is allowed to update the result in that memory address. 

In [72]:
function sum_thread_atomic(x)
    r = Atomic{eltype(x)}(zero(eltype(x)))
    @threads for i in eachindex(x)
        @inbounds atomic_add!(r, x[i])
    end
    return r[]
end

sum_thread_atomic(rand(100000))

50001.86747508756

Not every multithreading system works fastly and in an accurate manner! Sometimes they can make mess, since multiple threads can work on the same variable, performing operations that the designer do not want e.g. Atomi variables and functions make us fix this problem, but they are too slow.

There are fortunately other ways to make a workaround for this issue. For example we could store partial results indipendently for each thread in a temporary array, so that each thread updates only one element.

In [74]:
function threaded_sum_workaround(x)
    partial = zeros(eltype(x), nthreads())
    @threads for i in eachindex(x)
        @inbounds partial[threadid()] += x[i]
    end
    r = zero(eltype(x))
    for i in eachindex(partial)
        r += partial[i]
    end
    return r
end

threaded_sum_workaround (generic function with 1 method)

With this modality, we get the right results, with very fast time, since each thread updates only one element without any interferation on the work.

we can split the works between different threads by dividing different portions of a single array, in order to dedicate different portiosn of the array to different threads.

In [75]:
a = rand(1000000)

function sum_thread_split(A)
    r = Atomic{eltype(A)}(zero(eltype(A)))
    len, rem = divrem(length(A), nthreads())
    #Split the array equally among the threads
    @threads for t in 1:nthreads()
        r[] = zero(eltype(A))
        @simd for i in (1:len) .+ (t-1)*len
            @inbounds r[] += A[i]
        end
        atomic_add!(r, r[])
    end
    result = r[]
    #process up the remaining data
    @simd for i in length(A)-rem+1:length(A)
        @inbounds result += A[i]
    end
    return result
end

@btime sum_thread_split($a)

  19.515 ms (22 allocations: 2.22 KiB)


378080.3915203181

## Scheduling threads

Two of the most used scheduling patterns are quite common in Julia and can be achieved with `@threads`. 

__Static Scheduling__ is the mechanism where the order/way that the threads/processes are executing in the code is already controlled, by evenly dividing the amount of work among all available threads. If any synchronization primitive (locks, semaphores, joins, sleeps) has been used over threads in the program, then static scheduling is on. This is useful if workload is known a priori before code execution.  

__Dynamic Scheduling__ is the mechanism where thread scheduling is done by the operating systems based on any scheduling algorithm implemented in OS level. So the execution order of threads will be completely dependent on that algorithm, unless we have put some control on it. It is faster but often it is not thread-safe. This is useful when the exact workload of subtasks before execution is unknown.

In [82]:
a = rand(10000)  # Create array of random numbers
p = zeros(nthreads())  # Allocate a partial sum for each thread
# Threads macro splits the iterations of array `a` evenly among threads
@threads for x in a
   p[threadid()] += x  #Compute partial sums for each thread
end
s = sum(p)  # Compute the total sum = total workload

5044.834607021272

In [85]:
# we can see workload balancing by computing the ratio of workload per thread to the total workload.
println(round.(p/s, digits=3))

[0.247, 0.252, 0.251, 0.25]


In [87]:
function task()
    x = rand(0.001:0.001:0.05)  # Generate a variable workload
    return x  # Return the workload
end

n = 1000  # Number of tasks
p = zeros(Threads.nthreads())  # Total workload per thread
@sync for i in 1:n
   @spawn p[Threads.threadid()] += task()  # Spawn tasks and sum the workload
end
s = sum(p)

25.541999999999987

In [88]:
println(round.(p/s, digits=3))

[0.17, 0.739, 0.089, 0.001]


## Exercises


## __Q1__
The following function estimates π with n samples. 

- Parallelize the code using `@threads`and creating the function `estimatepi_threads(n)` (Hint: apart from modifying the function parallelize function call `estimates[i] = estimatepi_threads(10000000/nt)` in order to assign one thread to each i)  
- Take as estimated π value the mean computed with `Statistics` package
- Measure elapsed execution time for a big number of points and compute the obtained speedup.
- What happens if the number of threads increases? Will you get always higher speedups?
- π relative error for serial code is bigger or smaller than parallel code one? 

In [109]:
n = 10000000

function estimatepi(n)
    area_circle = 0
    @inbounds for i = 1:n
        x, y = rand(), rand();
        r = x^2 + y^2
        if r < 1.0
            area_circle +=1
        end
    end
    return 4* area_circle/n
end

@benchmark estimatepi(n)

BenchmarkTools.Trial: 124 samples with 1 evaluation.
 Range (min … max):  38.928 ms … 44.338 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     40.150 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   40.339 ms ±  1.048 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

        █                                                      
  ▃▁▄▆▄▆█▄▅▆▅▃▆▄▄▆▆▅▅▅▄▄▄▄▄▄▄▃▄▃▁▄▁▃▃▃▄▁▁▁▁▁▃▁▄▁▃▁▃▃▃▁▁▁▁▁▁▁▃ ▃
  38.9 ms         Histogram: frequency by time        43.7 ms <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [119]:
function estimatepi_threads(n::Int64)
    area_circle = 0 
    @threads for i = 1:n
        x = rand();
        y = rand();
        r = x^2 + y^2
        if r < 1.0
            @inbounds area_circle +=1
        end
    end
    return 4* area_circle/n
end    

@benchmark estimatepi_threads(n)

BenchmarkTools.Trial: 18 samples with 1 evaluation.
 Range (min … max):  257.217 ms … 388.269 ms  ┊ GC (min … max): 0.00% … 5.27%
 Time  (median):     284.059 ms               ┊ GC (median):    3.88%
 Time  (mean ± σ):   296.145 ms ±  33.153 ms  ┊ GC (mean ± σ):  3.02% ± 2.30%

  ▁    ▁█ ██ ▁ ▁  ▁    ▁ ▁▁ █                   ▁             ▁  
  █▁▁▁▁██▁██▁█▁█▁▁█▁▁▁▁█▁██▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  257 ms           Histogram: frequency by time          388 ms <

 Memory estimate: 119.79 MiB, allocs estimate: 7850412.

In [ ]:
nt = nthreads()
estimates = zeros(nt)

@threads for i=1:nt
    estimates[i] = estimatepi_threads(n/nt)
end

estpi_t = Statistics.mean(estimates)

# Distributed computing with Julia

Julia gives the possibility to start multiple indipendent processes, either on single host, or across a network. So we can control, communicate, and execute programs across an entire cluster

- Processes do not share any memory (heap or data), and can use multiple cores on the same machine or on different machines
- No Synchronization issues but we need to explicitly manage communication between processes, since they do not share memory! Communication takes place via messages  
- Processes are typically managed by the OS, whereas threads are managed by Julia's scheduler.
- Suitable for massive computational jobs

**The downside is that starting a process may be slow since each process requires much more memory than a multithread program.**

## The Master-Worker Model

There are several HPC models which distribute the amount of job to perform among different processors. Julia uses the so-called __master-work model__. In this model, we start with a single control process which is called master, controlling the worker processes. So the communication between Julia processes is __one-sided__ and the programmer needs to explicitly manage only one process in a two-processes operation.